In [1]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
import json
from datetime import datetime,timedelta
import os

env = "production"
provider = os.environ.get("CLOUD_PROVIDER", "aws")
# provider = "aws"
# provider = "gcp"
fs = "gs" if provider == "gcp" else "s3"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
216,application_1652123500543_0218,pyspark,idle,Link,Link,wenfeng,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##  zero-client

In [2]:

date_day = "2022-05-18"
date_hour = "*"
# date_hour = "00"
# band= "5"


s3_ap_stats = "{fs}://mist-data-science-dev/wenfeng/zero-clients_5_ap_stats_5/dt={date}/hr={hr}".format(fs=fs, date=date_day, hr=date_hour)
df_ap_radio = spark.read.parquet(s3_ap_stats)
    
# df_ap_radio=df_ap.withColumn("date_hour", F.from_unixtime(F.col('terminator_timestamp')/1_000_000, format='yyyy-MM-dd HH:ss'))\
#     .select("org_id", "site_id", "id", "hostname", "firmware_version", "model", "date_hour", "terminator_timestamp", F.explode("radios").alias("radio"))\
#     .filter('radio.band=={band} and radio.dev!="r2"'.format(band=band))\
#     .select("id", "date_hour","terminator_timestamp", "radio.*")\
#     .withColumn("num_wlans", F.size("wlans"))\
#     .withColumn("tx_errors_ratio", (F.col("tx_failed") + F.col("tx_retried"))/F.col("tx_pkts"))\
#     .withColumn("rx_errors_ratio", (F.col("rx_errors") + F.col("rx_dups"))/F.col("rx_pkts"))\
#     .orderBy(F.col("terminator_timestamp").asc())
        
# num_ap_stats = df_ap_radio.count()   


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_ap_radio.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- org_id: string (nullable = true)
 |-- site_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- date_hour: string (nullable = true)
 |-- terminator_timestamp: long (nullable = true)
 |-- dev: string (nullable = true)
 |-- band: string (nullable = true)
 |-- channel: integer (nullable = true)
 |-- secondary_channel: integer (nullable = true)
 |-- bandwidth: integer (nullable = true)
 |-- noise_floor: integer (nullable = true)
 |-- tx_power: integer (nullable = true)
 |-- max_tx_power: integer (nullable = true)
 |-- per_antenna_rssi: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- utilization_all: double (nullable = true)
 |-- utilization_tx: double (nullable = true)
 |-- utilization_rx_in_bss: double (nullable = true)
 |-- utilization_rx_other_bss: double (nullable = true)
 |-- utilization_unknown_wifi: double (nullable = true)
 |-- utilization_non_wifi: double (nullable = true)
 |-- dfs_radar_detected: boolean (nullable = true)
 |-

In [4]:
from pyspark.sql.window import Window
# *[F.lag(c, offset=shiftAmount).over(window).alias(c) for c in valueColumns]

prev_cols = ["date_hour", "terminator_timestamp", "num_clients", "channel", "bandwidth" ]
shiftAmount = -1 
window = Window.partitionBy(F.col('id'), F.col('dev')).orderBy(F.col('terminator_timestamp').desc())
df_ap_radio_t = df_ap_radio.select("*", *[F.lag(c, offset=shiftAmount).over(window).alias("prev_" +c) for c in prev_cols])\
    .withColumn("time_diff", F.col("terminator_timestamp") - F.col("prev_terminator_timestamp"))\
    .withColumn("num_client_diff", (F.col("num_clients") - F.col("prev_num_clients"))/1_000_000)\
    .withColumn("channel_updated", F.col("channel") != F.col("prev_channel"))\
    .withColumn("bandwidth_updated", F.col("bandwidth") != F.col("prev_bandwidth"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# df_ap_radio.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
%%pretty
test_cols = ["id", "dev"] + prev_cols + [ "prev_" + c for c in prev_cols] + ["time_diff", "num_client_diff", "channel_updated", "bandwidth_updated", "date_hour", "prev_date_hour" ]
df_ap_radio_t.select(test_cols).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

id,dev,date_hour,terminator_timestamp,num_clients,channel,bandwidth,prev_date_hour,prev_terminator_timestamp,prev_num_clients,prev_channel,prev_bandwidth,time_diff,num_client_diff,channel_updated,bandwidth_updated
5c-5b-35-3e-89-71,r1,2022-05-18 23:59:06,1652918346857825,0,48,20,2022-05-18 23:58:04,1652918284857644,0,48,20,62000181,0.0,false,false
5c-5b-35-3e-89-71,r1,2022-05-18 23:58:04,1652918284857644,0,48,20,2022-05-18 23:57:02,1652918222854764,0,48,20,62002880,0.0,false,false
5c-5b-35-3e-89-71,r1,2022-05-18 23:57:02,1652918222854764,0,48,20,2022-05-18 23:56:00,1652918160860154,0,48,20,61994610,0.0,false,false
5c-5b-35-3e-89-71,r1,2022-05-18 23:56:00,1652918160860154,0,48,20,2022-05-18 23:54:58,1652918098853034,0,48,20,62007120,0.0,false,false
5c-5b-35-3e-89-71,r1,2022-05-18 23:54:58,1652918098853034,0,48,20,2022-05-18 23:53:56,1652918036858795,0,48,20,61994239,0.0,false,false
5c-5b-35-3e-89-71,r1,2022-05-18 23:53:56,1652918036858795,0,48,20,2022-05-18 23:52:54,1652917974882061,0,48,20,61976734,0.0,false,false
5c-5b-35-3e-89-71,r1,2022-05-18 23:52:54,1652917974882061,0,48,20,2022-05-18 23:51:54,1652917914859098,0,48,20,60022963,0.0,false,false
5c-5b-35-3e-89-71,r1,2022-05-18 23:51:54,1652917914859098,0,48,20,2022-05-18 23:50:52,1652917852877467,0,48,20,61981631,0.0,false,false
5c-5b-35-3e-89-71,r1,2022-05-18 23:50:52,1652917852877467,0,48,20,2022-05-18 23:49:50,1652917790892627,0,48,20,61984840,0.0,false,false
5c-5b-35-3e-89-71,r1,2022-05-18 23:49:50,1652917790892627,0,48,20,2022-05-18 23:48:48,1652917728857540,0,48,20,62035087,0.0,false,false


In [7]:
# df_ap_radio_t.filter("")
# # 1652834161780704 - 1652834223763132
# 61989994/1_000_000

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## client-dropping

In [8]:
s3 = "s3://mist-data-science-dev/wenfeng/clients-dropping_5/dt=2022-05-25/hr=000"
df_suspicious_radios = spark.read.parquet(s3)
df_suspicious_radios.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- org_id: string (nullable = true)
 |-- site_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- firmware_version: string (nullable = true)
 |-- model: string (nullable = true)
 |-- terminator_timestamp: long (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- date_hour: string (nullable = true)
 |-- dev: string (nullable = true)
 |-- band: string (nullable = true)
 |-- channel: integer (nullable = true)
 |-- secondary_channel: integer (nullable = true)
 |-- bandwidth: integer (nullable = true)
 |-- noise_floor: integer (nullable = true)
 |-- tx_power: integer (nullable = true)
 |-- max_tx_power: integer (nullable = true)
 |-- per_antenna_rssi: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- utilization_all: double (nullable = true)
 |-- utilization_tx: double (nullable = true)
 |-- utilization_rx_in_bss: double (nullable = true)
 |-- utilization_rx_other_bss: double (nullable = true

In [9]:
# df_suspicious_radios = df
df_suspicious_radios.select("num_clients", "prev_num_clients", "num_client_diff", "bcn_per_wlan", "prev_bcn_per_wlan", "bcn_drop").summary().show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+------------------+--------------------+------------------+-----------------+-------------------+
|summary|num_clients|  prev_num_clients|     num_client_diff|      bcn_per_wlan|prev_bcn_per_wlan|           bcn_drop|
+-------+-----------+------------------+--------------------+------------------+-----------------+-------------------+
|  count|      47032|             47032|               47032|             47032|            47032|              47032|
|   mean|        0.0| 4.947269943868005|  -4.947269943868E-6| 566.2097791779586|578.4632551064599|-12.253475939219735|
| stddev|        0.0|3.1932080033097514|3.193208003309751E-6|103.51693321548495|75.20503222691706|  72.63918497676958|
|    min|          0|                 4|            -1.41E-4|               0.0|              0.0|          -602.3333|
|    25%|          0|                 4|             -5.0E-6|             582.0|            582.0|              -15.0|
|    50%|          0|                 4|        

In [10]:
df_suspicious_radios= df_suspicious_radios\
        .withColumn("bcn_drop_large", F.col("bcn_drop")>100)\
        .withColumn("noise_floor_high", F.col("noise_floor") > -65.0) \
        .withColumn("tx_phy_err_high", F.col("tx_phy_err") > 1.0) \
        .withColumn("tx_error_high", (F.col("tx_failed") + F.col("tx_retried"))/ F.col("tx_pkts") >0.1) \
        .withColumn("rx_error_high", (F.col("rx_dups") + F.col("rx_errors"))/ F.col("rx_pkts")>0.1) \
        .withColumn("utilization_non_wifi_high", F.col("utilization_non_wifi") > 0.20)

cols = ["bcn_drop_large", "channel_updated", "bandwidth_updated","noise_floor_high","utilization_non_wifi_high" ,
                            "tx_phy_err_high", "tx_error_high", "rx_error_high"]
df_suspicious_radios_g = df_suspicious_radios.select(cols).groupBy(cols).count().orderBy(cols)

nrows = df_suspicious_radios_g.count()
df_suspicious_radios_g.show(nrows, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------------+-----------------+----------------+-------------------------+---------------+-------------+-------------+-----+
|bcn_drop_large|channel_updated|bandwidth_updated|noise_floor_high|utilization_non_wifi_high|tx_phy_err_high|tx_error_high|rx_error_high|count|
+--------------+---------------+-----------------+----------------+-------------------------+---------------+-------------+-------------+-----+
|false         |false          |false            |false           |false                    |false          |null         |null         |485  |
|false         |false          |false            |false           |false                    |false          |null         |false        |85   |
|false         |false          |false            |false           |false                    |false          |null         |true         |15   |
|false         |false          |false            |false           |false                    |false          |false        |null         

In [11]:
df_suspicious_radios_1 = df_suspicious_radios.filter("not bcn_drop_large and tx_phy_err_high")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## scope

In [12]:
df_org = spark.read.parquet("s3://mist-secorapp-production/dimension/org")\
    .select(F.col("id").alias("OrgID"),F.col("name").alias("org_name"))
df_name = spark.read.parquet("s3://mist-secorapp-production/dimension/site/site.parquet")\
    .select(F.col("id").alias("SiteID"),F.col("name").alias("site_name"),F.col("org_id").alias("OrgID")).join(df_org,["OrgID"])\
                .select("org_name","site_name","OrgID","SiteID")

t_org_1 = df_name.select(['OrgID','org_name']).withColumnRenamed('OrgID', 'org_id').dropDuplicates()
t_org_2 = df_name.select(['SiteID','site_name']).withColumnRenamed('SiteID', 'site_id').dropDuplicates()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:


def ap_scope(df_query, df_name):
    if 'org_name' not in df_query.columns:
        df_query= df_query.join(df_name, df_query.site_id == df_name.SiteID, how='left')

    df_query.select('org_id', 'org_name').groupBy("org_id", 'org_name').count().orderBy(F.col("count").desc()).show(truncate=False)

    df_query.select('org_id','org_name', 'site_id', 'site_name')\
        .groupBy("org_id", 'org_name', 'site_id', 'site_name')\
        .count()\
        .orderBy(F.col("count").desc()).show(truncate=False)


    df_query.select('org_id','org_name', 'site_id', 'site_name', 'id').groupBy('org_id','org_name', 'site_id', 'site_name', 'id')\
        .count().orderBy(F.col("count").desc())\
        .show(truncate=False)

    # %%pretty 
    df_query.select('model').groupBy('model').count().orderBy(F.col("count").desc()).show(truncate=False)

    df_query.select('model', 'firmware_version').groupBy('model', 'firmware_version').count().orderBy(F.col("count").desc()).show(truncate=False)
    
    return df_query

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_suspicious_radios=ap_scope(df_suspicious_radios, df_name)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+-------------------------------------+-----+
|org_id                              |org_name                             |count|
+------------------------------------+-------------------------------------+-----+
|56de201d-e63b-4312-9858-40f4cfe35c7f|AmazonOTFC-Prod                      |24756|
|5a578f3c-f094-4d13-a7dd-96f0aaecb5b6|AmazonOTN19-Prod                     |3179 |
|4f7ce014-2148-4d41-88f0-07c9c2e6f16c|CFA-Comcast                          |3165 |
|d8cec22e-e0c2-11e5-8d0f-02e208b2d34f|AmazonProd                           |1843 |
|a8c903ae-e6d0-42a7-b815-fe395dea8017|T-Mobile USA Retail                  |1065 |
|a15b4c50-378e-4103-974e-ea9014cd281b|Bunnings                             |1013 |
|05ecd967-81cf-4928-b8ba-ffd767fc19bd|Coppell ISD                          |711  |
|38a18d4d-1623-4985-86d0-1bb06e5e2a48|UPS OPS                              |696  |
|39b04356-4cc3-41cc-bc1a-5de54beb1941|Meijer                               |622  |
|5f7

In [15]:
df_suspicious_radios=ap_scope(df_suspicious_radios_1, df_name)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+-------------------------------+-----+
|org_id                              |org_name                       |count|
+------------------------------------+-------------------------------+-----+
|d8cec22e-e0c2-11e5-8d0f-02e208b2d34f|AmazonProd                     |8    |
|5a578f3c-f094-4d13-a7dd-96f0aaecb5b6|AmazonOTN19-Prod               |3    |
|e091512a-528f-414b-941d-3e5876d03510|Centre For Neuro Skills        |2    |
|83610c38-ee38-4519-99df-740b91bca429|VISITS TECHNOLOGIES, INC.      |1    |
|64bfc882-18f6-4cdb-a301-03cdba26744e|Holy AG                        |1    |
|b9cb46f2-a4e9-49fd-93af-1557a5531389|istyle Inc.                    |1    |
|1f1dcc2c-a7f8-4bc1-8408-4bf339d7e1a2|Connectivity Source            |1    |
|490a1abb-c1a4-4861-b7e7-fb5299ae2b9d|Henderson County Public Schools|1    |
|56de201d-e63b-4312-9858-40f4cfe35c7f|AmazonOTFC-Prod                |1    |
|a7cc1161-cb41-4e5d-aa41-118a1f06cd12|Petco                          |1    |